In [ ]:

import pandas as pd
from connection import connect

In [ ]:
SCHEMA='production'
co_oltp,etl_conn,etl_conn_or=connect(SCHEMA)

In [ ]:
dim_fecha = pd.DataFrame({
    "date": pd.date_range(start='1/1/2005', end='31/12/2014', freq='D')
})

In [ ]:
dim_fecha['day_number_of_week'] = ((dim_fecha['date'].dt.day_of_week + 1) % 7) + 1
dim_fecha['day_number_of_month']=dim_fecha['date'].dt.day
dim_fecha['day_number_of_year']=dim_fecha['date'].dt.day_of_year
dim_fecha['week_number_of_year'] = dim_fecha['date'].apply(
    lambda d: ((d - pd.Timestamp(d.year, 1, 1)).days + pd.Timestamp(d.year, 1, 1).weekday() + 1) // 7 + 1
)
dim_fecha['month_number_of_year']=dim_fecha['date'].dt.month
dim_fecha['calendar_quarter']=dim_fecha['date'].dt.quarter
dim_fecha['calendar_year']=dim_fecha['date'].dt.year
dim_fecha['calendar_semester']=(((dim_fecha['date'].dt.month-1)//6)+1)
dim_fecha['fiscal_year']=dim_fecha['date'].apply(lambda d: d.year if d.month <=7 else d.year+1)
dim_fecha['fiscal_month']=dim_fecha['date'].apply(lambda d:((d.month)-7)%12+1)
dim_fecha['fiscal_quarter']=((dim_fecha['fiscal_month']-1)//3) + 1
dim_fecha['fiscal_semester']=((dim_fecha['fiscal_month']-1)//6) + 1

dim_fecha.drop('fiscal_month',inplace=True,axis=1)

#languages

# TRAER ESTO DE UTILS
english_days = {
    1: 'Sunday',
    2: 'Monday',
    3: 'Tuesday',
    4: 'Wednesday',
    5: 'Thursday',
    6: 'Friday',
    7: 'Saturday'
}

spanish_days = {
    1: 'Domingo',
    2: 'Lunes',
    3: 'Martes',
    4: 'Miércoles',
    5: 'Jueves',
    6: 'Viernes',
    7: 'Sábado'
}

french_days = {
    1: 'Dimanche',
    2: 'Lundi',
    3: 'Mardi',
    4: 'Mercredi',
    5: 'Jeudi',
    6: 'Vendredi',
    7: 'Samedi'
}

english_months = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

spanish_months = {
    1: 'Enero',
    2: 'Febrero',
    3: 'Marzo',
    4: 'Abril',
    5: 'Mayo',
    6: 'Junio',
    7: 'Julio',
    8: 'Agosto',
    9: 'Septiembre',
    10: 'Octubre',
    11: 'Noviembre',
    12: 'Diciembre'
}

french_months = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai',
    6: 'Juin',
    7: 'Juillet',
    8: 'Août',
    9: 'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}




dim_fecha['english_day_name_of_week'] = dim_fecha['day_number_of_week'].map(english_days)
dim_fecha['spanish_day_name_of_week']=dim_fecha['day_number_of_week'].map(spanish_days)
dim_fecha['french_day_name_of_week']=dim_fecha['day_number_of_week'].map(french_days)

dim_fecha['english_month_name']=dim_fecha['month_number_of_year'].map(english_months)
dim_fecha['spanish_month_name']=dim_fecha['month_number_of_year'].map(spanish_months)
dim_fecha['french_month_name']=dim_fecha['month_number_of_year'].map(french_months)


dim_fecha











In [ ]:
dim_fecha['date_key']=dim_fecha['date'].dt.strftime('%Y%m%d').astype(int)


dim_fecha.rename(columns={'date':'full_date_alternate_key'},inplace=True)


dim_fecha

## Load

In [20]:
dim_fecha.to_sql('dim_date',etl_conn,if_exists='append',index=False)

652